In [33]:
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 300)
pd.options.mode.chained_assignment = None  # default='warn'

In [21]:
YEAR = '2021'

In [22]:
df1 = pd.read_csv('data/to_extract/database-c-find_'+YEAR+'_1.csv')
df2 = pd.read_csv('data/to_extract/database-c-find_'+YEAR+'_2.csv')
df3 = pd.read_csv('data/to_extract/database-c-find_'+YEAR+'_3.csv')
df4 = pd.read_csv('data/to_extract/database-c-find_'+YEAR+'_4.csv')
df = pd.concat([df1,df2,df3,df4])
#df.to_csv('data/to_extract/database-c-find_PRODUCTION-'+YEAR+'.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/to_extract/database-c-find_2021_1.csv'

In [ ]:
df = pd.read_csv('data/to_extract/database-c-find_PRODUCTION-'+YEAR+'.csv')
#df['StudyDate'] = pd.to_datetime(df['StudyDate'])
df['StudyDate'] = pd.to_datetime(df['StudyDate'], format='%Y%m%d')
display(df.StudyDate.dt.month.value_counts())

In [ ]:
#df[['PatientID','StudyInstanceUID']].to_csv('config/TO_EXTRACT/'+YEAR+'.csv')

In [ ]:
#df.groupby(['PatientID','StudyDate','StudyTime']).first().to_csv('data/to_extract/CARDIO_REPORT_2020.csv')

### Start Here

In [34]:
YEAR = '2021'

In [35]:
import os


def list_files(dir):
    r = []
    subdirs = [x[0] for x in os.walk(dir)]
    for subdir in subdirs:
        files = os.walk(subdir).__next__()[2]
        
        if len(files) > 0:
            for file in files:
                r.append(os.path.join(subdir, file))
    return r

In [36]:
r = list_files('/media/data1/ravram/DeepCORO/'+YEAR)
display(len(r), " files found")
df = pd.DataFrame(list(zip(r)),
               columns =['path'])
new_2 = df["path"].str.split("/", n=8, expand=True)
df["year"] = new_2[5]
df["StudyInstanceUID"] = new_2[7]
df["patient_id_anon"] = new_2[6]
df["dicom_id"] = new_2[8]

#except:
#    print("No path")

141059

' files found'

In [37]:
display("Displaying the DICOMS to extract for year %%", YEAR)
df_to_extract = pd.read_csv('config/TO_EXTRACT/'+YEAR+'.csv')
#display(df_to_extract.loc[df_to_extract['PatientID']==570066].head(n=5))
display(df_to_extract.describe())

'Displaying the DICOMS to extract for year %%'

'2021'

,PatientID
count,123.000000
mean,462287.300813
std,163269.336213
min,11025.000000
25%,324732.000000
50%,548107.000000
75%,591272.000000
max,799995.000000


In [41]:
#df_extracted = pd.read_csv('data/to_extract/database-c-find_PRODUCTION-2017.csv')
#df_to_extract_2 = pd.read_csv('data/to_extract/database-c-find_PRODUCTION_2020_2.csv')
#df_to_extract = pd.concat([df_to_extract_1, df_to_extract_2])
#df_extracted_m = pd.merge(df_extracted, df, how='inner', on=['StudyInstanceUID'])
#df_extracted_m.to_csv('../CathAI/data/DeepCORO/EXAMS_Extracted/2017_dicom_extracted.csv')

In [42]:
df_extracted = pd.DataFrame({'path':r})
df_extracted['FileType'] = df_extracted.path.apply(lambda x: x.split('.')[-1])
display("Total files", df_extracted.FileType.value_counts())
df_extracted = df_extracted.loc[df_extracted['FileType']=='dcm']
new_2 = df_extracted["path"].str.split("/", n=8, expand=True)
df_extracted['mrn'] = new_2[6]
df_extracted['exam_id'] = new_2[7]
df_extracted['dicom_id'] = new_2[8].str.rstrip('.dcm')
df_extracted_study_level = df_extracted.groupby(['mrn','exam_id']).first().reset_index()
display(df_extracted_study_level.head(n=5))

'Total files'

dcm          141056
completed         1
whole             1
failed            1
Name: FileType, dtype: int64

,mrn,exam_id,path,FileType,dicom_id
0,100141617087339680314641590245863839451,2.16.840.1.113883.3.234.1.3.101.1.2.1053.2021.5920901.2,/media/data1/ravram/DeepCORO/2021/100141617087339680314641590245863839451/2.16.840.1.113883.3.234.1.3.101.1.2.1053.2021.5920901.2/2.16.840.1.113883.3.234.1.3.101.1.2.1053.2021.5920901.2.77777.dcm,dcm,2.16.840.1.113883.3.234.1.3.101.1.2.1053.2021.5920901.2.77777
1,100173129898864044757031908835634854696,2.16.124.113611.1.118.1.1.5790577,/media/data1/ravram/DeepCORO/2021/100173129898864044757031908835634854696/2.16.124.113611.1.118.1.1.5790577/1.2.392.200036.9116.4859.20210517145945831.1.112.dcm,dcm,1.2.392.200036.9116.4859.20210517145945831.1.112
2,100178611806377453478479688295608744355,1.3.46.670589.28.11411431729720210312184732794995,/media/data1/ravram/DeepCORO/2021/100178611806377453478479688295608744355/1.3.46.670589.28.11411431729720210312184732794995/1.3.46.670589.28.114114317297.2021031221045729727.3.dcm,dcm,1.3.46.670589.28.114114317297.2021031221045729727.3
3,100196407450277947908554130408792721913,2.16.124.113611.1.118.1.1.5799496,/media/data1/ravram/DeepCORO/2021/100196407450277947908554130408792721913/2.16.124.113611.1.118.1.1.5799496/1.3.12.2.1107.5.4.5.147598.30000021052611373995100000034.dcm,dcm,1.3.12.2.1107.5.4.5.147598.30000021052611373995100000034
4,100197317241274317410675080458196097822,2.16.124.113611.1.118.1.1.5677650,/media/data1/ravram/DeepCORO/2021/100197317241274317410675080458196097822/2.16.124.113611.1.118.1.1.5677650/1.3.12.2.1107.5.4.5.135214.30000020122815173402300000185.dcm,dcm,1.3.12.2.1107.5.4.5.135214.30000020122815173402300000185


In [43]:
display(df_extracted_study_level.mrn.value_counts())
display(df_to_extract.tail(n=5))
display(df_extracted.tail(n=5))

257012871274156347697735973404592210538    9
23389078990667372365950792610356311630     7
54298432197473416668542362197360201309     7
225570121026747532462060279198697961313    7
72546341879166899615257779062629767140     7
                                          ..
312317916595163411309322979923522485444    1
320374666138133736201638367141275927401    1
286392981917607310978699737822831141863    1
248153632242468289839828546048516226412    1
5446138980753582196568318781810155545      1
Name: mrn, Length: 7337, dtype: int64

,PatientID,StudyInstanceUID
118,599306,2.16.124.113611.1.118.1.1.5978991
119,599364,2.16.124.113611.1.118.1.1.5979179
120,520484,2.16.124.113611.1.118.1.1.5979239
121,599408,2.16.124.113611.1.118.1.1.5979283
122,324536,2.16.124.113611.1.118.1.1.5993628


,path,FileType,mrn,exam_id,dicom_id
141054,/media/data1/ravram/DeepCORO/2021/57512104842499975397846939886651683259/2.16.124.113611.1.118.1.1.5979239/1.3.12.2.1107.5.4.5.135214.30000021121411131593700000369.dcm,dcm,57512104842499975397846939886651683259,2.16.124.113611.1.118.1.1.5979239,1.3.12.2.1107.5.4.5.135214.30000021121411131593700000369
141055,/media/data1/ravram/DeepCORO/2021/57512104842499975397846939886651683259/2.16.124.113611.1.118.1.1.5979239/1.3.12.2.1107.5.4.5.135214.30000021121411131593700000375.dcm,dcm,57512104842499975397846939886651683259,2.16.124.113611.1.118.1.1.5979239,1.3.12.2.1107.5.4.5.135214.30000021121411131593700000375
141056,/media/data1/ravram/DeepCORO/2021/57512104842499975397846939886651683259/2.16.124.113611.1.118.1.1.5979239/1.2.276.0.7230010.3.1.3.477015473.7684.1639503020.182.dcm,dcm,57512104842499975397846939886651683259,2.16.124.113611.1.118.1.1.5979239,1.2.276.0.7230010.3.1.3.477015473.7684.1639503020.182
141057,/media/data1/ravram/DeepCORO/2021/57512104842499975397846939886651683259/2.16.124.113611.1.118.1.1.5979239/1.3.12.2.1107.5.4.5.135214.30000021121411131593700000385.dcm,dcm,57512104842499975397846939886651683259,2.16.124.113611.1.118.1.1.5979239,1.3.12.2.1107.5.4.5.135214.30000021121411131593700000385
141058,/media/data1/ravram/DeepCORO/2021/57512104842499975397846939886651683259/2.16.124.113611.1.118.1.1.5979239/1.3.12.2.1107.5.4.5.135214.30000021121411131593700000346.dcm,dcm,57512104842499975397846939886651683259,2.16.124.113611.1.118.1.1.5979239,1.3.12.2.1107.5.4.5.135214.30000021121411131593700000346


In [44]:
display("To extract prior to removal of already extracted exams", df_to_extract.describe())
df_extracted = df_to_extract.loc[df_to_extract['StudyInstanceUID'].isin(df_extracted_study_level['exam_id'])]
df_to_be_extracted = df_to_extract.loc[~df_to_extract['StudyInstanceUID'].isin(df_extracted_study_level['exam_id'])]

display("Extracted", df_extracted.describe())
display("To be extracted", df_to_be_extracted.describe())
display(df_to_be_extracted.head(n=5))

'To extract prior to removal of already extracted exams'

,PatientID
count,123.000000
mean,462287.300813
std,163269.336213
min,11025.000000
25%,324732.000000
50%,548107.000000
75%,591272.000000
max,799995.000000


'Extracted'

,PatientID
count,66.00000
mean,466847.30303
std,164968.90853
min,11025.00000
25%,325843.25000
50%,555210.50000
75%,590777.50000
max,600311.00000


'To be extracted'

,PatientID
count,57.000000
mean,457007.298246
std,162580.710172
min,36341.000000
25%,324928.000000
50%,514575.000000
75%,591498.000000
max,799995.000000


,PatientID,StudyInstanceUID
2,799995,2.16.124.113611.1.118.1.1.5681024
3,583114,2.16.124.113611.1.118.1.1.5683643
4,459846,1.2.840.113711.687442.6.26120.634504188.26.2116281012.10
5,379859,2.16.124.113611.1.118.1.1.5683530
6,585288,1.2.840.113711.687442.6.11964.633623173.26.2116281012.10


In [45]:
df_to_be_extracted = df_to_be_extracted[['PatientID','StudyInstanceUID']]
df_to_be_extracted = df_to_be_extracted.set_index('PatientID')
df_to_be_extracted.to_csv('config/TO_EXTRACT/'+YEAR+'.csv')
extract_filename = 'config/TO_EXTRACT/'+YEAR+'.csv'
print("Extracted new file for DICOM copying", extract_filename)

Extracted new file for DICOM copying config/TO_EXTRACT/2021.csv


In [46]:
#df_to_be_extracted = pd.read_csv('config/TO_EXTRACT/'+YEAR+'.csv')
#display(df_to_be_extracted)

## Test Anonymization

In [1]:
path = '/media/data1/ravram/DeepCORO/2019/267920477097029320671587129404529631902/1.3.6.1.4.1.23849.3598784133.16.1636881522360890000/1.3.6.1.4.1.23849.3340572284.16.1636881522434990000.dcm'

In [2]:
import pydicom
dataset = pydicom.dcmread(path)

In [3]:
display(dataset)

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 186
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: X-Ray Angiographic Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.6.1.4.1.23849.1751036954.16.1636881522434990000
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.40.0.13.1.1.1
(0002, 0013) Implementation Version Name         SH: 'dcm4che-1.4.38'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['DERIVED', 'PRIMARY', 'SINGLE PLANE', 'SINGLE A', 'STORE FLUORO']
(0008, 0016) SOP Class UID                       UI: X-Ray Angiographic Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.23849.1751036954.16.1636881522